In [1]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import gemini_analyze_fuction
import gemini_comment_fuction
import gemini_setup_function
import gemini_summary_function
import get_and_crawl_data
import plotly_and_upload

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(gemini_analyze_fuction)
importlib.reload(gemini_comment_fuction)
importlib.reload(gemini_setup_function)
importlib.reload(gemini_summary_function)
importlib.reload(get_and_crawl_data)
importlib.reload(plotly_and_upload)

from import_default import *
from import_database import *
from import_other import *
from gemini_analyze_fuction import *
from gemini_comment_fuction import *
from gemini_setup_function import *
from gemini_summary_function import *
from get_and_crawl_data import *
from plotly_and_upload import *

#### Lấy dữ liệu Mongo

In [2]:
date_series = get_mongo_collection(ref_db, 'date_series')
time_series = get_mongo_collection(ref_db, 'time_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')

In [3]:
projection = {
    "_id": 0,
    "date": 1,
    "ticker": 1,
    "open": 1,
    "high": 1,
    "low": 1,
    "close": 1,
    "pct_change": 1,
    "diff": 1,
    "volume": 1,
    "option": 1,
    "SMA_20": 1,
    "SMA_60": 1,
    "RSI_14": 1,
    "week_open": 1,
    "month_prev_high": 1,
    "month_prev_low": 1,
    "month_open": 1,
    "quarter_prev_high": 1,
    "quarter_prev_low": 1,
    "quarter_open": 1,
    "year_open": 1,
    "MFIBO_0382": 1,
    "MFIBO_0500": 1,
    "MFIBO_0618": 1,
    "QFIBO_0382": 1,
    "QFIBO_0500": 1,
    "QFIBO_0618": 1,
	"YFIBO_0382": 1,
    "YFIBO_0500": 1,
    "YFIBO_0618": 1,
}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

In [4]:
other_ticker_df = get_mongo_collection(stock_db, 'other_ticker', projection=projection)
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

#### Tạo các models cho gemini

In [5]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))

# Sắp xếp danh sách model theo thứ tự ổn định
# fast_model_list = select_fast_models(get_gemini_models())
# standard_model_list = select_standard_models(get_gemini_models())
fast_model_list = ['gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']
standard_model_list = ['gemini-2.5-flash', 'gemini-2.0-flash', 'gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

#### Lấy dữ liệu lịch sử VN30F1M

In [ ]:
# Đọc tất cả các file FiinProX từ thư mục Downloads
fiinprox_files = glob.glob(os.path.join(str(Path.home() / "Downloads"), "*FiinProX*.xlsx"))
# Xoá đi các file không chứa dữ liệu Tự Doanh
for file in fiinprox_files:
    td_check = pd.read_excel(file).applymap(lambda x: isinstance(x, str) and "Thống kê thị trường_Tự doanh-Chi tiết" in x).any().any()
    if not td_check:
        fiinprox_files.remove(file)
if len(fiinprox_files) == 0:
    raise FileNotFoundError("❌ Chưa tải dữ liệu Tự Doanh từ FiinProX.")

# Sắp xếp fiinprox_files theo thời gian tạo (tăng dần) và lấy file mới nhất
last_fiinprox_file = sorted(fiinprox_files, key=os.path.getctime)[-1]

#Đọc dữ liệu Tự Doanh từ file mới nhất
td_vn30f1m_df = pd.read_excel(last_fiinprox_file, skiprows=8).iloc[:, :12]
td_vn30f1m_df = td_vn30f1m_df.dropna().iloc[:20].reset_index(drop=True)
td_vn30f1m_df = td_vn30f1m_df.drop(td_vn30f1m_df.columns[[3, 5, 7, 9]], axis=1)
td_vn30f1m_df.columns = ['ticker', 'date', 'buy_volume', 'sell_volume', 'buy_value', 'sell_value', 'net_value', 'net_volume']
td_vn30f1m_df[[col for col in td_vn30f1m_df.columns if '_value' in col]] = td_vn30f1m_df[[col for col in td_vn30f1m_df.columns if '_value' in col]]/1000
td_vn30f1m_df['type'] = 'TD'

#Kiểm tra xem dữ liệu Tự Doanh đã mới nhất chưa
td_date_check = td_vn30f1m_df['date'].max()
date_to_check = date_series.iloc[2].item()
if td_date_check <= date_to_check:
    raise ValueError(f"❌ Dữ liệu Tự Doanh không mới nhất. Ngày trong dữ liệu: {td_date_check}, ngày yêu cầu: {date_to_check}")
else:
    print(f"✅ Đã cập nhật dữ liệu Tự Doanh đã mới nhất. Ngày trong dữ liệu: {td_date_check}")

#Lưu riêng ticker cho VN30F1M dùng sau này
vn30f1m_ticker = td_vn30f1m_df['ticker'].unique()[0]

#Dữ liệu NN từ Fireant
nn_vn30f1m_df = nntd_index_df[(nntd_index_df['ticker'] == 'VN30F1M') & (nntd_index_df['type'] == 'NN')].reset_index(drop=True)
nn_vn30f1m_df['ticker'] = vn30f1m_ticker

#Ghép bảng NN và TD
daily_8h45_nntd_df = pd.concat([nn_vn30f1m_df, td_vn30f1m_df], axis=0, ignore_index=True)

✅ Đã cập nhật dữ liệu Tự Doanh đã mới nhất. Ngày trong dữ liệu: 2025-07-29 00:00:00


#### Vẽ biểu đồ kĩ thuật

In [63]:
vn30f1m_chart_df = full_index_df[full_index_df['ticker'] == 'VN30F1M'].iloc[:120].sort_values('date').reset_index(drop=True)
vn30f1m_chart_df['ticker'] = vn30f1m_ticker
image_name = 'TA_DAILY_VN30F1M.png'

line_name_dict = {
    'SMA_20': 'SMA 20',
    'SMA_60': 'SMA 60',
    'week_open': 'WEEK OPEN',
    'month_open': 'MONTH OPEN',
    'quarter_open': 'QUARTER OPEN',
    'year_open': 'YEAR OPEN',
    'month_prev_high': 'LAST MHIGH',
    'month_prev_low': 'LAST MLOW',
    'quarter_prev_high': 'LAST QHIGH',
    'quarter_prev_low': 'LAST QLOW',
    'MFIBO_0382': 'MFIBO 0.382',
    'MFIBO_0500': 'MFIBO 0.500',
    'MFIBO_0618': 'MFIBO 0.618',
    'QFIBO_0382': 'QFIBO 0.382',
    'QFIBO_0500': 'QFIBO 0.500',
    'QFIBO_0618': 'QFIBO 0.618',
	'YFIBO_0382': 'YFIBO 0.382',
    'YFIBO_0500': 'YFIBO 0.500',
    'YFIBO_0618': 'YFIBO 0.618'
}

chart_config = create_chart_config(
	title_font_size=30,
	axis_font_size=24,
	tag_font_size=24,
	price_tag_font_size=24,
    min_spacing_ratio = 0.042,
	margin=dict(l=20, r=220, t=20, b=20, pad=10)
)

max_attempts = 5
for attempt in range(max_attempts):
    try:
        fig, fig_images = create_financial_chart(
            vn30f1m_chart_df, # DataFrame của bạn
            width=1400,
            height=1200,
            line_name_dict=line_name_dict,
            line_columns=list(line_name_dict.keys()),
            chart_config=chart_config,
            path='../output/cts_weekly_chart/',
            symbol_name='VN30F1M',
            image_name = image_name
        )
        upload_to_r2(fig_images, image_name)
        break
    except Exception as e:
        if attempt == max_attempts - 1:
            raise e
# fig.show()

In [98]:
def daily_vn30f1m_comment_prompt(df):
    prompt = f"""
Đây là dữ liệu của chỉ số hợp đồng tương lai VN30F1M:
    {df.to_csv(index=False, sep='|', lineterminator='\n')}
**Vai trò:** Bạn là một **nhà phân tích kỹ thuật cấp cao**, chuyên về giao dịch phái sinh, đưa ra nhận định **chiến thuật hàng ngày** cho nhà đầu tư.
**Nhiệm vụ:** Dựa trên dữ liệu và các chỉ báo kỹ thuật, hãy viết một báo cáo phân tích và xây dựng chiến lược giao dịch 2 chiều cho **phiên kế tiếp**, tuân thủ nghiêm ngặt cấu trúc **10 câu, 4 đoạn**.
**Yêu cầu về nội dung, cấu trúc và định dạng:**
    Báo cáo phải gồm **10 câu, mỗi câu dài khoảng 20 đến 25 từ** và được chia thành **4 đoạn văn riêng biệt**. Bắt đầu thẳng vào nội dung, không có lời chào.
    **Đoạn 1 (3 câu): Phân tích Phiên Gần Nhất**
        * **Câu 1:** Nhận định tổng quan về diễn biến giá trong **phiên giao dịch gần nhất (ngày dd/mm)**.
        * **Câu 2:** Phân tích vai trò của hai đường **SMA 20 và SMA 60** trong việc xác nhận xu hướng hiện tại.
        * **Câu 3:** Đánh giá chỉ báo **RSI 14**, đặc biệt là các tín hiệu quá mua/quá bán và ý nghĩa của chúng cho phiên tới.
    **Đoạn 2 (3 câu): Xác định các Vùng giá Cốt lõi**
        * **Câu 4:** Phân tích mốc hỗ trợ/kháng cự quan trọng theo **khung Tuần** (kết hợp O-H-L gần nhất và Fibonacci gần nhất).
        * **Câu 5:** Phân tích mốc hỗ trợ/kháng cự quan trọng theo **khung Tháng** (kết hợp O-H-L gần nhất và Fibonacci gần nhất).
        * **Câu 6:** Tổng hợp và nhấn mạnh **một vùng hỗ trợ và một vùng kháng cự then chốt nhất** sẽ quyết định xu hướng **trong các phiên tới**.
    **Đoạn 3 (2 câu): Kịch bản cho Vị thế LONG
        * **Câu 7:** Trình bày các điểm vào lệnh tiềm năng cho vị thế LONG, gộp cả kịch bản vào lệnh LONG khi giá bứt phá kháng cự và khi giá điều chỉnh về hỗ trợ.
        * **Câu 8: Xác định một mục tiêu chốt lời (TP) chung cho cả hai kịch bảnvà xác định lợi nhuận dữ kiến và các ngưỡng cắt lỗ (SL) riêng biệt cho từng kịch bản SHORT (ví dụ: '...với các ngưỡng cắt lỗ tương ứng tại X và Y')
    **Đoạn 4 (2 câu): Kịch bản cho Vị thế SHORT
        * **Câu 9:** Tương tự, trình bày các điểm vào lệnh cho vị thế SHORT, bao gồm kịch bản vào lệnh SHORT tại kháng cự và khi giá phá vỡ hỗ trợ.
        * **Câu 10:** Thiết lập một mục tiêu chốt lời (TP) chung và xác định lợi nhuận dữ kiến và các ngưỡng cắt lỗ (SL) riêng biệt cho từng kịch bản SHORT (ví dụ: '...với các ngưỡng cắt lỗ tương ứng tại X và Y').
**Yêu cầu về ngôn ngữ và trình bày:**
    * **YÊU CẦU VỀ THUẬT NGỮ (CỰC KỲ QUAN TRỌNG): Luôn sử dụng đúng thuật ngữ phái sinh "vị thế LONG" và "vị thế SHORT". TUYỆT ĐỐI KHÔNG sử dụng các từ đồng nghĩa như "mua", "bán", "kịch bản mua", "vị thế bán", "phe mua", "phe bán" trong toàn bộ báo cáo.**
    * **RÀNG BUỘC CHIẾN LƯỢC NGẮN HẠN:** Tất cả các điểm vào lệnh, chốt lời (TP) và cắt lỗ (SL) đề xuất phải nằm trong một phạm vi hợp lý. Khoảng cách từ điểm vào lệnh đến TP hoặc SL không nên vượt quá 50 điểm.
    * **LƯU Ý QUAN TRỌNG:** Các khoảng cách cho TP và SL (bằng điểm) phải được tính từ **ĐIỂM VÀO LỆNH ĐỀ XUẤT**, không phải từ giá đóng cửa hiện tại.
    * **Văn phong:** Sắc bén, khách quan, quyết đoán và đa dạng, tránh lặp lại cấu trúc câu.
    * **TUYỆT ĐỐI KHÔNG** được viết tên cột dữ liệu gốc và **KHÔNG** có lời chào.
    * **LUÔN LUÔN** sử dụng ngày dạng dd/mm trong bài viết.
    """
    return prompt

ta_vn30f1m_comment = generate_content_with_model_dict(standard_model_dict, daily_vn30f1m_comment_prompt(vn30f1m_chart_df), 'daily_vn30f1m_comment')


✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vn30f1m_comment.
